In [2]:
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "/home/rahul/Downloads/675484_1187790_bundle_archive/Face Mask Dataset/Train"
test_dir="/home/rahul/Downloads/675484_1187790_bundle_archive/Face Mask Dataset/Test"


train_with_mask= train_dir +"/WithMask"
test_with_mask=test_dir + "/WithMask"
train_without_mask= train_dir + "/WithoutMask"
test_without_mask= test_dir +"/WithoutMask"


# To convert RGB images into array of numbers
data_generator = ImageDataGenerator(rescale = 1.0/255.0, zoom_range = 0.2)  


batch_size = 32
training_data = data_generator.flow_from_directory(directory = train_dir,
                                                   target_size = (64, 64),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')
testing_data = data_generator.flow_from_directory(directory = test_dir,
                                                  target_size = (64, 64),
                                                  batch_size = batch_size,
                                                  class_mode = 'binary')


from tensorflow.keras.applications import vgg16

vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape= training_data.image_shape)

from keras import layers, models, optimizers
model = models.Sequential()
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

vgg_conv.trainable = False

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

history = model.fit(training_data,
                        steps_per_epoch = 100,
                        epochs = 20,
                        validation_data = testing_data,
                        validation_steps = int(1000/batch_size)
                             )



Found 10000 images belonging to 2 classes.
Found 992 images belonging to 2 classes.
Epoch 1/20
100/100 [==============================] - 147s 1s/step - loss: 0.2052 - accuracy: 0.9149 - val_loss: 0.2012 - val_accuracy: 0.9194
Epoch 2/20
100/100 [==============================] - 148s 1s/step - loss: 0.1486 - accuracy: 0.9463 - val_loss: 0.1657 - val_accuracy: 0.9355
Epoch 3/20
100/100 [==============================] - 144s 1s/step - loss: 0.1305 - accuracy: 0.9494 - val_loss: 0.1118 - val_accuracy: 0.9577
Epoch 4/20
100/100 [==============================] - 144s 1s/step - loss: 0.1134 - accuracy: 0.9563 - val_loss: 0.0947 - val_accuracy: 0.9688
Epoch 5/20
100/100 [==============================] - 144s 1s/step - loss: 0.1102 - accuracy: 0.9582 - val_loss: 0.0890 - val_accuracy: 0.9657
Epoch 6/20
100/100 [==============================] - 145s 1s/step - loss: 0.1165 - accuracy: 0.9572 - val_loss: 0.0947 - val_accuracy: 0.9647
Epoch 7/20
100/100 [==============================] - 144s

In [3]:
model.save("face_mask.tf")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: face_mask.tf/assets


In [17]:
testing_image("/home/rahul/Downloads/675484_1187790_bundle_archive/Face Mask Dataset/Test/WithMask/51.png")

[[0.]]


'With Mask'

In [9]:
from tensorflow.keras.preprocessing import image
import numpy as np
def testing_image(image_directory):
    test_image = image.load_img(image_directory, target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(x = test_image)
    print(result)
    if result[0][0]  == 1:
        prediction = 'Without Mask'
    else:
        prediction = 'With Mask'
    return prediction

